In [ ]:
!pip install langchain-community
!pip install tiktoken
!pip install faiss-cpu
#!pip install openai
#!pip install langchain
!pip install lancedb
!pip install openai==0.28
#!pip install huggingface_hub
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import pandas as pd

df = pd.read_csv('/content/train (2).csv')

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
missing_values = df.isnull().sum()

missing_values

,0
id,0
person_age,0
person_income,0
person_home_ownership,0
person_emp_length,0
loan_intent,0
loan_grade,0
loan_amnt,0
loan_int_rate,0
loan_percent_income,0


In [ ]:
import sqlite3

loan = 'loan-approval.db'

In [ ]:
conn = sqlite3.connect(loan)

In [ ]:
# Create a cursor object
cursor = conn.cursor()

In [ ]:
df.to_sql('loan-data', conn, if_exists='append', index=False)

58645

# Use LLM

In [ ]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI # Import ChatOpenAI from langchain

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
openai_api_key = os.environ['OPENAI_API_KEY']

llm = OpenAI()
model = ChatOpenAI()

<ipython-input-12-af61a004926b>:11: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
<ipython-input-12-af61a004926b>:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

db = SQLDatabase.from_uri('sqlite:///loan-approval.db')

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent_executor.run("Describe the first 10 rows from table")

<ipython-input-13-461bbe3c34ef>:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("Describe the first 10 rows from table")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: loan-data I should query the loan-data table schema to see the column names.
Action: sql_db_schema
Action Input: loan-data
CREATE TABLE "loan-data" (
	id INTEGER, 
	person_age INTEGER, 
	person_income INTEGER, 
	person_home_ownership TEXT, 
	person_emp_length REAL, 
	loan_intent TEXT, 
	loan_grade TEXT, 
	loan_amnt INTEGER, 
	loan_int_rate REAL, 
	loan_percent_income REAL, 
	cb_person_default_on_file TEXT, 
	cb_person_cred_hist_length INTEGER, 
	loan_status INTEGER
)

/*
3 rows from loan-data table:
id	person_age	person_income	person_home_ownership	person_emp_length	loan_intent	loan_grade	loan_amnt	loan_int_rate	loan_percent_income	cb_person_default_on_file	cb_person_cred_hist_length	loan_status
0	37	35000	RENT	0.0	EDUCATION	B	6000	11.49	0.17	N	14	0
1	22	56000	OWN	6.0	MEDICAL	C	4000	13.35	0.07	N	2	0
2	29	28800	OWN	8.0	PERSONAL	A	6000	8.9	0.21	N	10	0
*/ I should query the first 10 rows from the loan-da

"The first 10 rows from the loan-data table are [(37, 35000, 'RENT', 0.0, 'EDUCATION', 'B', 6000, 11.49, 0.17, 'N', 14, 0), (22, 56000, 'OWN', 6.0, 'MEDICAL', 'C', 4000, 13.35, 0.07, 'N', 2, 0), (29, 28800, 'OWN', 8.0, 'PERSONAL', 'A', 6000, 8.9, 0.21, 'N', 10, 0), (30, 70000, 'RENT', 14.0, 'VENTURE', 'B', 12000, 11.11, 0.17, 'N', 5, 0), (22, 60000, 'RENT', 2.0, 'MEDICAL', 'A', 6000, 6.92, 0.1, 'N', 3, 0), (27, 45000"

In [ ]:
tools = [tool for tool in toolkit.get_tools()]

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent_executor.run('how much people apply loan for medical purpose?')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:loan-data
I should look at the schema for the loan-data table.
Action: sql_db_schema
Action Input: loan-data
CREATE TABLE "loan-data" (
	id INTEGER, 
	person_age INTEGER, 
	person_income INTEGER, 
	person_home_ownership TEXT, 
	person_emp_length REAL, 
	loan_intent TEXT, 
	loan_grade TEXT, 
	loan_amnt INTEGER, 
	loan_int_rate REAL, 
	loan_percent_income REAL, 
	cb_person_default_on_file TEXT, 
	cb_person_cred_hist_length INTEGER, 
	loan_status INTEGER
)

/*
3 rows from loan-data table:
id	person_age	person_income	person_home_ownership	person_emp_length	loan_intent	loan_grade	loan_amnt	loan_int_rate	loan_percent_income	cb_person_default_on_file	cb_person_cred_hist_length	loan_status
0	37	35000	RENT	0.0	EDUCATION	B	6000	11.49	0.17	N	14	0
1	22	56000	OWN	6.0	MEDICAL	C	4000	13.35	0.07	N	2	0
2	29	28800	OWN	8.0	PERSONAL	A	6000	8.9	0.21	N	10	0
*/
Now that I know the correct table and columns, I can construct a

'10934'

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    max_iterations=20,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent_executor.run('What is the minimum income required to be eligible for a loan, and what other factors are considered in loan approval decisions, based on the provided dataset?')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: loan-data Now that I have the list of tables, I can query the schema of the loan-data table to see what columns are available.
Action: sql_db_schema
Action Input: loan-data
CREATE TABLE "loan-data" (
	id INTEGER, 
	person_age INTEGER, 
	person_income INTEGER, 
	person_home_ownership TEXT, 
	person_emp_length REAL, 
	loan_intent TEXT, 
	loan_grade TEXT, 
	loan_amnt INTEGER, 
	loan_int_rate REAL, 
	loan_percent_income REAL, 
	cb_person_default_on_file TEXT, 
	cb_person_cred_hist_length INTEGER, 
	loan_status INTEGER
)

/*
3 rows from loan-data table:
id	person_age	person_income	person_home_ownership	person_emp_length	loan_intent	loan_grade	loan_amnt	loan_int_rate	loan_percent_income	cb_person_default_on_file	cb_person_cred_hist_length	loan_status
0	37	35000	RENT	0.0	EDUCATION	B	6000	11.49	0.17	N	14	0
1	22	56000	OWN	6.0	MEDICAL	C	4000	13.35	0.07	N	2	0
2	29	28800	OWN	8.0	PERSONAL	A	6000	8.9	0.21	N	10	0
*/

'The minimum income required to be eligible for a loan is $4,200 and the other factors considered are loan intent, loan grade, person home ownership, and credit history.'

Prompt Engineering

In [ ]:
create_sql_agent?

In [ ]:
prompt_prefix = """
##Instructions:
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
As part of your final answer, ALWAYS include an explanation of how to got to the final answer, including the SQL query you run. Include the explanation and the SQL query in the section that starts with "Explanation:".

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don\'t know" as the answer.

##Tools:

"""

prompt_format_instructions = """
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.
"""

In [ ]:
agent_executor = create_sql_agent(
    prefix=prompt_prefix,
    format_instructions = prompt_format_instructions,
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=10
)

result = agent_executor.run("What are the most age apply loan")



> Entering new SQL Agent Executor chain...
Action: sql_db_schema
Action Input: loan_infoError: table_names {'loan_info'} not found in database Since the error message tells me that the table does not exist, I should check the tables in the database.
Action: sql_db_list_tables
Action Input:loan-data I found a table called "loan-data" which sounds relevant.
Action: sql_db_schema
Action Input: loan-data
CREATE TABLE "loan-data" (
	id INTEGER, 
	person_age INTEGER, 
	person_income INTEGER, 
	person_home_ownership TEXT, 
	person_emp_length REAL, 
	loan_intent TEXT, 
	loan_grade TEXT, 
	loan_amnt INTEGER, 
	loan_int_rate REAL, 
	loan_percent_income REAL, 
	cb_person_default_on_file TEXT, 
	cb_person_cred_hist_length INTEGER, 
	loan_status INTEGER
)

/*
3 rows from loan-data table:
id	person_age	person_income	person_home_ownership	person_emp_length	loan_intent	loan_grade	loan_amnt	loan_int_rate	loan_percent_income	cb_person_default_on_file	cb_person_cred_hist_length	loan_status
0	37	35000	R

In [ ]:
from IPython.display import Markdown, HTML, display
def printmd(string):
    display(Markdown(string))
print(result)

The most common age for applying for a loan is 37 years old, with a loan intent of "PERSONAL". The SQL query used to obtain this information was: SELECT person_age, loan_intent FROM "loan-data" ORDER BY person_age DESC LIMIT 10.


In [ ]:
agent_executor = create_sql_agent(
    prefix=prompt_prefix,
    format_instructions = prompt_format_instructions,
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=10
)

result = agent_executor.run("What is the maximum interest rate offered for loans intended for 'home_improvement'?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:
loan-data I should query the schema for loan-data
Action: sql_db_schema
Action Input: loan-data
CREATE TABLE "loan-data" (
	id INTEGER, 
	person_age INTEGER, 
	person_income INTEGER, 
	person_home_ownership TEXT, 
	person_emp_length REAL, 
	loan_intent TEXT, 
	loan_grade TEXT, 
	loan_amnt INTEGER, 
	loan_int_rate REAL, 
	loan_percent_income REAL, 
	cb_person_default_on_file TEXT, 
	cb_person_cred_hist_length INTEGER, 
	loan_status INTEGER
)

/*
3 rows from loan-data table:
id	person_age	person_income	person_home_ownership	person_emp_length	loan_intent	loan_grade	loan_amnt	loan_int_rate	loan_percent_income	cb_person_default_on_file	cb_person_cred_hist_length	loan_status
0	37	35000	RENT	0.0	EDUCATION	B	6000	11.49	0.17	N	14	0
1	22	56000	OWN	6.0	MEDICAL	C	4000	13.35	0.07	N	2	0
2	29	28800	OWN	8.0	PERSONAL	A	6000	8.9	0.21	N	10	0
*/ I have all the information I need to construct my query.  I should query the

In [ ]:
from IPython.display import Markdown, HTML, display
def printmd(string):
    display(Markdown(string))
print(result)

12.99
